# Erhebung und Analyse von Sensordaten - Time of flight Kamera


## Step 1: Load data
Get the input path and verify it is a valid file

In [1]:
import utilities


input_file_paths = utilities.load_input_file_paths()
data_sets = utilities.load_depth_data_sets(input_file_paths)


Found 22 files in Messdaten
Data set contains 107 frames
Data set contains 169 frames
Data set contains 103 frames
Data set contains 136 frames
Data set contains 125 frames
Data set contains 114 frames
Data set contains 113 frames
Data set contains 329 frames
Data set contains 123 frames
Data set contains 104 frames
Data set contains 110 frames
Data set contains 124 frames
Data set contains 122 frames
Data set contains 173 frames
Data set contains 131 frames
Data set contains 111 frames
Data set contains 123 frames
Data set contains 100 frames
Data set contains 114 frames
Data set contains 126 frames
Data set contains 114 frames
Data set contains 147 frames


In [ ]:
import os

for file_name, frames in data_sets.items():
    file_name = (os.path.splitext(file_name)[0] + '.gif').replace(" ", "_")
    utilities.create_gif_from_frames(frames, file_name)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/dkdoseiffermann/Documents/Uni/5th_semester/erfassung_und_analyse_von_sensordaten/output/Messdaten/2022-05-04 09%3A56%3A08.227366_Terabee_measurement.gif'

<Figure size 640x480 with 0 Axes>

Load the input data into a pandas dataframe of width 80 * height 60

Print the frame as a heat map using matplotlib

In [ ]:
# from PIL import Image

# frames = []

# for i in range(np_array.shape[0]):
#     # Create the plot
#     plt.imshow(np_array[i], cmap='hot', interpolation='nearest', vmin=0, vmax=3321)
#     plt.axis('off')  # Hide axes for a cleaner look

#     # Save the plot as an image in memory
#     plt.tight_layout()
#     plt.draw()

#     # Convert the plot to a PIL image and add to frames list
#     image = Image.frombytes('RGB', plt.gcf().canvas.get_width_height(),
#                             plt.gcf().canvas.tostring_rgb())
#     frames.append(image)

#     plt.clf()  # Clear the figure for the next frame

# # Save frames as a GIF
# frames[0].save('heatmap_animation.gif', format='GIF', append_images=frames[1:],
#                save_all=True, duration=100, loop=0)